---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [8]:
import pandas as pd
import numpy as np

def blight_model():
    
    from sklearn.naive_bayes import BernoulliNB # to start, go for a crude binary feat. set
    # try p130 example SVC + MinMaxScaler or LR here instead.
    # use PCA with a visualisation also (p140)
    from adspy_shared_utilities import plot_class_regions_for_classifier
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier 
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import GridSearchCV
        
    from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc, classification_report

    # import the CSVs to Pandas Dfs
    train = pd.read_csv('train.csv', encoding="latin1")
    test = pd.read_csv('test.csv', encoding="latin1")
    
    # TODO: clean up the addresses - seems important
    addresses = pd.read_csv('addresses.csv', encoding="latin1")
    
    
    # adjust any cols and null vals etc
    # Drop the violators which classed "not responsible"; also remove NaN fine amount
    train.dropna(subset=['judgment_amount', 'compliance'], inplace=True)
    
    # make the amounts better money data type:
    train.judgment_amount = train.judgment_amount * 100.0
    train.judgment_amount = train.judgment_amount.astype(int)
    test.judgment_amount = test.judgment_amount * 100.0
    test.judgment_amount = test.judgment_amount.astype(int)
    
    train.discount_amount = train.discount_amount * 100.0
    train.discount_amount = train.discount_amount.astype(int)
    test.discount_amount = test.discount_amount * 100.0
    test.discount_amount = test.discount_amount.astype(int)
    
    # simplify compliance
    train.compliance = train.compliance.astype(int)
    
    # remove certain features
    columns = ['agency_name','inspector_name','violator_name','violation_zip_code',\
                'violation_street_number','violation_street_name',\
                'mailing_address_str_number','mailing_address_str_name','city',\
                'state', 'zip_code','non_us_str_code','country',\
                'clean_up_cost','disposition','grafitti_status',\
                'ticket_issued_date','hearing_date','fine_amount',\
                'violation_code','violation_description','admin_fee','state_fee','late_fee']
    
    for column in columns:
        test.drop(column, axis=1, inplace=True)
        train.drop(column, axis=1, inplace=True)
        
    # remove some extra train only columns:
    columns = ['payment_status','payment_date','payment_amount','balance_due']
    for column in columns:
        train.drop(column, axis=1, inplace=True)
        
    
    # required for return result:
    train = train.set_index('ticket_id')
    test = test.set_index('ticket_id')
    
    # use train for all training:
    y_train = train['compliance'] 
    
    # then remove these columns from remaining training data:
    columns = ['compliance', 'compliance_detail', 'collection_status']
    for column in columns:
        train.drop(column, axis=1, inplace=True)
    
    # reuse the train as X_train
    X_train = train.copy()
    
    # fit models:
    
    # try a logistic regression model
    # clf = LogisticRegression(C=0.01, class_weight='balanced') 
    
    # try gradient boost (should be better than random forest)
    clf = GradientBoostingClassifier(max_depth=5, learning_rate=0.25)
    
    clf.fit(X_train, y_train)

    # check the accuracy on the training set
    print(clf.score(X_train, y_train))
    
    # test predictions:
    predicted = clf.predict(test)
    
    # input the test to get probabilities for submission of assignment:
    probs = clf.predict_proba(test)[:,1]
    
    # make Pandas series of probs ...with index to return: as per shape needed (test.csv)
    probs = pd.Series(probs, index=test.index)
    return probs

blight_model()





/Users/Luke/anaconda/envs/py35data/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2831: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0.935426569927


ticket_id
284932    0.060770
285362    0.023534
285361    0.069594
285338    0.060770
285346    0.069594
285345    0.060770
285347    0.057381
285342    0.376677
285530    0.023534
284989    0.031176
285344    0.057381
285343    0.023534
285340    0.023534
285341    0.057381
285349    0.069594
285348    0.060770
284991    0.031176
285532    0.031176
285406    0.031176
285001    0.031176
285006    0.023534
285405    0.023534
285337    0.031176
285496    0.057381
285497    0.060770
285378    0.023534
285589    0.031176
285585    0.060770
285501    0.069594
285581    0.023534
            ...   
376367    0.036959
376366    0.036160
376362    0.036160
376363    0.060770
376365    0.036959
376364    0.036160
376228    0.036160
376265    0.036160
376286    0.305986
376320    0.036160
376314    0.036160
376327    0.305986
376385    0.305986
376435    0.498396
376370    0.305986
376434    0.057381
376459    0.069594
376478    0.000038
376473    0.036160
376484    0.017953
376482    0.031176
37